In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn import svm

from sklearn import datasets
file_path='smsspamcollection/SMSSpamCollection.txt'
data=pd.read_csv(file_path, delimiter='\t',header=None,skipinitialspace=True)

In [2]:
data=data.dropna(axis=0, how='any')

In [3]:
def pre_processing(sms):
    _str=re.sub("[^a-zA-Z]"," ",sms)
    words=_str.lower().split()
    new_str=[w for w in words if not w in set(stopwords.words("english"))]
    return (" ".join(new_str))

In [4]:
clean_data = []
for i in range(len(data[1])):
    clean_data.append(pre_processing(data[1][i]))
data[1] = clean_data

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test = train_test_split(data.iloc[:,1], data.iloc[:,0],test_size=0.1, random_state=50)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(analyzer = "word",tokenizer = None,preprocessor = None,stop_words = None,max_features = 5000)
x_train = vectorizer.fit_transform(x_train).toarray()
x_test = vectorizer.transform(x_test).toarray()

In [7]:
# SVM
clf=svm.SVC(kernel='linear',C=1.0)
clf.fit(x_train,y_train)

predicted=clf.predict(x_test)
accuracy=np.mean(predicted==y_test)
print (accuracy)

0.987455197133


In [8]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(x_train, y_train)

y_pred=neigh.predict(x_test)
from sklearn import metrics
score_test = metrics.accuracy_score(y_test, y_pred)
print(score_test)

0.940860215054


In [9]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
nb_pred = gnb.fit(x_train, y_train).predict(x_test)
accuracy = np.mean(nb_pred==y_test)
print(accuracy)

0.890681003584


In [10]:
# Neural network - multi layer
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
mlp.fit(x_train, y_train)
ann_pred = mlp.predict(x_test)
score = np.mean(ann_pred==y_test)
print(score)

0.987455197133


In [11]:
from sklearn.model_selection import GridSearchCV
param_grid = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']}]
svm_c = svm.SVC()
train = GridSearchCV(svm_c, param_grid,cv=5)
train.fit(x_train,y_train)
print(train.best_params_)

{'kernel': 'linear', 'C': 1}
